# **PCSE** GMS Invoice **/ Xero Automation** v2.0
`code by Jan du Plessis`<BR>

>**GitHub** https://github.com/janduplessis883<BR>
**Email** drjanduplessis@icloud.com<BR>
**Twitter** @janduplessis<BR>


1.   To use this Automation **Save copy to DRIVE** form the File menu.
2.   On **first use**, follow the instructions and **RUN** each code sell individually.
3.   Once configured, speed up invoice processing by using **Run All** cells from the Runtime menu.





# Installing required **Python modules**

In [ ]:
!pip install PyPDF2

# Extracting **text** from PDF Invoice.

Download GMS Invoice from [PSCE Online](https://pcse.england.nhs.uk/). Download **Extended PDF version** to your hard drive.<BR>
**Upload PDF** to Colabs enviroment - see left panel (folder icon).<BR>
Right click file in left panel, select **Copy Path**<BR>
Paste **File Path** prompted, when the next code segment runs.
## ✋ **INPUT REQUIRED** in the next code cell.

In [ ]:
from PyPDF2 import PdfReader

# Collect required information
#invoice_pdf_net = float(input("Enter GMS Payment received acc. Bank Statement?: £"))
invoice_date_input = input("What is the invoice date? Format: 'DD-MMM-YYYY' ")
invoice_number_input = input("Specify unique Xero Invoice Number: AutoINV-")
invoice_number_input = "AutoINV-" + invoice_number_input
pdf_path = input("Paste (ctrl V) 'file path' to invoice PDF: ")

# Open the PDF file using the PdfReader class
with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PdfReader(pdf_file)

    # Access the pages in the PDF file using the pages attribute
    page_one = pdf_reader.pages[0]
    page1 = page_one.extract_text()
    page_two = pdf_reader.pages[1]
    page2 = page_two.extract_text()

# Close the PDF file
pdf_file.close()

print('PDF extraction successful.')


What is the invoice date? Format: 'DD-MMM-YYYY' 23-Nov-2023
Specify unique Xero Invoice Number: AutoINV-004003
Paste (ctrl V) 'file path' to invoice PDF: /content/GppPayments.pdf
PDF extraction successful.


# **Parsing** invoice info from text.

In [ ]:
#Prepare Python List from extracted data - Check Current Format
list_all = page1 + page2
list_all2 = list_all.splitlines()

#print("Output > list_all2 ", type(list_all2), "Successful")
print(list_all)

GPP Payments
Statement Date : 20/11/2023
Contractor ID : E87750
Net : £68,905.09
Invoice NHSE : £1,790.68
Credit NHSE : £0.00
Invoice COM : £74,812.49
Credit COM : -£7,698.08
Statement : November - The Surgery (E87750)
Ref. no : 543680
Description 
GMS/PMS/APMS Contract Value 
GMS/PMS/APMS Contract Value 
Calculation of Monthly Global Sum Payment 428,343.68 
Temporary Residents Adjustment 2,107.77 Add
London Weighting Allowance 9,134.20 Add
Superannuation Premium 0.00 Add
Appraisal Premium 0.00 Add
Adjusted Global Sum Annual Payment ------------- 
439,585.65 
Equals 
Initial Global Sum Monthly Payment 36,632.14 
Additional Service Opt Outs 0.00 Minus 
Capitation Monthly Payment GMS/PMS/APMS (OCT, NOV & DEC) ------------- 
36,632.14 
Additonal Opt Out Details 
Total  0.00  
Capitation Data (GP Practice) 
Main Surgery Postcode SW59JA 
Raw Practice List Size 4,220 
Count of Patients Registered in Past Twelve Months 709
Patients Average distance to main surgery (metres) 272.84121 
Practice

In [ ]:
# The phrase to remove
phrase_to_remove = 'Quality Outcomes Framework'

# Use a list comprehension to create a new list with the phrase removed from each string
modified_list = [my_string.replace(phrase_to_remove, '') for my_string in list_all2]

# Print the modified list
print(modified_list)

['GPP Payments', 'Statement Date : 20/11/2023', 'Contractor ID : E87750', 'Net : £68,905.09', 'Invoice NHSE : £1,790.68', 'Credit NHSE : £0.00', 'Invoice COM : £74,812.49', 'Credit COM : -£7,698.08', 'Statement : November - The Surgery (E87750)', 'Ref. no : 543680', 'Description ', 'GMS/PMS/APMS Contract Value ', 'GMS/PMS/APMS Contract Value ', 'Calculation of Monthly Global Sum Payment 428,343.68 ', 'Temporary Residents Adjustment 2,107.77 Add', 'London Weighting Allowance 9,134.20 Add', 'Superannuation Premium 0.00 Add', 'Appraisal Premium 0.00 Add', 'Adjusted Global Sum Annual Payment ------------- ', '439,585.65 ', 'Equals ', 'Initial Global Sum Monthly Payment 36,632.14 ', 'Additional Service Opt Outs 0.00 Minus ', 'Capitation Monthly Payment GMS/PMS/APMS (OCT, NOV & DEC) ------------- ', '36,632.14 ', 'Additonal Opt Out Details ', 'Total  0.00  ', 'Capitation Data (GP Practice) ', 'Main Surgery Postcode SW59JA ', 'Raw Practice List Size 4,220 ', 'Count of Patients Registered in P

In [ ]:
import re

# Assuming you have a list like this
my_list = ["Another string with £1200.50Quality Outcomes Framework",
           "£1235678.99Quality",
           "Yet another string without the phrase"]

# Define a regular expression pattern to match numbers and a decimal point after the pound sign
pattern = r'£(\d+(\.\d+)?)'

# Use a list comprehension to extract the matched numbers
extracted_numbers = [re.search(pattern, my_string).group(1) if re.search(pattern, my_string) else None for my_string in my_list]

# Print the extracted numbers
print(extracted_numbers)

['1200.50', '1235678.99', None]


Removing no-invoicable data - append to **final_list**.<BR>
Finding **Global Sum** Amount.

In [ ]:
#Extract Total Paid to variable
total_paid = list_all2[-1]
total_amount = float(total_paid.split('£')[-1].replace(",",""))
print("Total Paid - ", total_amount)

counta = 0
final_list = []
exclude_list = ["Paycode Description", "GMC Code Name Tier Rate", "Paycode Description", "Type Source", "Total Amount Paid", "Invoice", "Net", "Credit", "Percentage Received",  'GMC Code Name Month']

iter_index = 0
# first iter over list_all2 with list comprehension to find index of Global sum +1 and extract for proceeding.
for list in list_all2:
  if [ex for ex in exclude_list if(ex in list)]:
    #print("Invoice Line Removed - ", iter_index)
    iter_index = iter_index +1

  elif "£" in list:
    final_list.append(str(list))
    counta = counta +1
    iter_index = iter_index +1

# Find Index of Global Sum ammount in all_list2
iter_index = 0
search_text = "Capitation Monthly Payment"
for list in list_all2:
  if search_text not in list:
    iter_index = iter_index + 1

  else:
    print("Global Sum Amount found - list_all2 index: " + str(iter_index + 1))
    global_sum_index = iter_index + 1

globalsum_amount = list_all2[global_sum_index]
globalsum_amount = globalsum_amount.rstrip().replace(",","")
print(globalsum_amount)
global_sum_str = "Global Sum £"+ str(globalsum_amount)
#print(global_sum_str)
final_list.append(global_sum_str)

#print("final_list created - entries = " + str(len(final_list)))
print(final_list)
print("Global Sum - **Successful**")

Total Paid -  68905.09
Global Sum Amount found - list_all2 index: 24
36632.14
['GSUMG COM £2,513.85', 'GSUMG COM £2,486.88', 'RANDR 2023-2024 ICBRates 01/05/23 to 31/03/24 M COM £3,834.83Quality Outcomes Framework ', 'Aspiration COM £2,217.12', 'QUACHG PY 2022/23 new adj value Atrial Fibrillation (AF) QOF Achvemnt A COM £11.90', 'MENCBG Men B - GMS NHSE £10.06', 'SHINGG Shingles Immunisation (GMS) NHSE £20.12', 'ZPERTG Pertussis (Whooping Cough) NHSE £20.12', 'PNEUMG Pneumoc Imm DES (GMS) NHSE £30.18', 'MMRVAG Measles Mumps and Rubella (MMR) Vaccinations (GMS) NHSE £40.24', 'CDFLUG Childhood Flu DES (GMS) NHSE £50.30', 'SIXVAG C&M-GMS - 6 in 1 Vaccination NHSE £50.30', 'INFLUG Influenza DES (GMS) NHSE £1,569.36', 'DESPRG GMS DES Participation in the PCN COM £592.51', 'ZPRCAG LBS End of Life Apr-Mar24 M COM £144.68', 'ZPRCAG LBS Support For Carers Apr-Mar24 M COM £649.37', 'ZPRCAG LBS MCMW Apr-Mar24 M COM £1,862.47', 'ZPRCAG LBS Access Apr-Mar24 M COM £2,271.11', 'NRENT 23/24 Notional R

# Prepare data for **DataFrame**
* Splitting invoice description and amount.
* **Append** column lists for dataframe.

In [ ]:
import re

# Assuming you have a string like this
my_string = "135555666.99Qutcome"

# Define a regular expression pattern to match numbers and a decimal point
pattern = r'(\d+(\.\d+)?)'

# Use re.search to find the pattern in the string
result = re.search(pattern, my_string)

# Extract the matched numbers
extracted_numbers = result.group(1) if result else None

# Print the extracted numbers
print(extracted_numbers)

135555666.99


In [ ]:
import re
pattern = r'(\d+(\.\d+)?)'

#Declairing required variables.
contact_name = []
email_address = []
PO1 = []
PO2 = []
PO3 = []
PO4 = []
city = []
PO_region = []
postal_code = []
country = []
address1 = []
address2 = []
address3 = []
address4 = []
sa_city = []
sa_region = []
sa_postalcode = []
sa_country = []
invoice_number = []
ref = []
invoice_date = []
due_date = []
planned_date = []
total = []
tax_total = []
invoice_amount_paid = []
invoice_amount_due = []
inv_item_code = []
description = []
quantitiy = []
unit_amount = []
line_amount = []
account_code = []
tax_type = []
discount =[]
tax_amount = []
trackn1 = []
tracko1 = []
trackn2 = []
tracko2 = []
currency = []
type_ = []
sent = []
status = []

#function to assign Account Code

account_code_dic = {
    "rates": "102",
    "childhood flu": "124A",
    "influenza": "124",
    "mmr": "137",
    "pneumoc": "125",
    "rotavirus": "128",
    "hpv": "158",
    "men b": "135",
    "hib/men c": "135A",
    "6 in 1": "120",
    "pms cis": "209",
    "last phase of life": "165",
    "hard to reach": "193",
    "carers": "193",
    "my way": "183B",
    "access": "193",
    "long covid": "193",
    "pcn participation": "250B",
    "rent": "111",
    "prescribing": "203",
    "ecg": "185",
    "spoke": "132A",
    "warfarin": "181",
    "homeless": "186",
    "wound": "192",
    "mental": "187",
    "diabetes": "184",
    "phlebotomy": "189",
    "coordinate": "183",
    "levy": "523",
    "global": "100",
    "aspiration": "103"
}

#function to assign account_code
def get_account_no(invoice_description):
  output = ""
  for x, y in account_code_dic.items():
    if x in invoice_description:
      output = y
    else:
      pass
  return output


for l in final_list:
  if "-------------" in l:

    index_in_l = final_list.index("-----------")
    print("****** Global Sum on next String **********", index_in_l)

    #value_pair_list = list.split("£")
    #net_income_str = value_pair_list[-1]
    #remove ',' from str
    #net_income = net_income_str.replace(",", "")
    #net_income = float(net_income)
    #print("Net Income = ", net_income, type(net_income))

  elif "-£" in l:
    print('invoiceable MINUS - '+ l)

    value_pair_list = l.split("-£")
    invoicable_desc = value_pair_list[0]
    invcoice_desc_lower = invoicable_desc.lower()
    value_pair_list = l.split("£")
    invoicable_desc = l[0]
    invoicable_value_str = value_pair_list[-1].replace(",","")
    result = re.search(pattern, invoicable_value_str)
    extracted_numbers = result.group(1) if result else None
    invoicable_value = float(extracted_numbers)

    contact_name.append('NHS England GMS')
    email_address.append("")
    PO1.append("")
    PO2.append('')
    PO3.append('')
    PO4.append('')
    city.append('')
    PO_region.append('')
    postal_code.append('')
    country.append('')
    address1.append('')
    address2.append('')
    address3.append('')
    address4.append('')
    sa_city.append('')
    sa_region.append('')
    sa_postalcode.append('')
    sa_country.append('')
    invoice_number.append(invoice_number_input)
    ref.append('')
    invoice_date.append(invoice_date_input)
    due_date.append(invoice_date_input)
    planned_date.append('')
    tax_total.append(0)
    invoice_amount_paid.append(0)
    inv_item_code.append('')
    description.append(invoicable_desc)
    quantitiy.append(1)
    unit_amount.append(invoicable_value)
    invoice_amount_due.append(invoicable_value)
    discount.append('')
    line_amount.append(invoicable_value)
    tax_type.append('No VAT')
    tax_amount.append(0)
    trackn1.append('')
    tracko1.append('')
    trackn2.append('')
    tracko2.append('')
    currency.append('GBP')
    type_.append('Sales Invoice')
    sent.append('')
    status.append('')
    account_code.append(get_account_no(invcoice_desc_lower))

  else:
    print("invoiceable POS - "+ l)

    value_pair_list = l.split("£")
    invoicable_desc = value_pair_list[0]
    invcoice_desc_lower = invoicable_desc.lower()
    invoicable_value_str = value_pair_list[-1].replace(",","")
    result = re.search(pattern, invoicable_value_str)
    extracted_numbers = result.group(1) if result else None
    invoicable_value = float(extracted_numbers)


    contact_name.append('NHS England GMS')
    email_address.append("")
    PO1.append("")
    PO2.append('')
    PO3.append('')
    PO4.append('')
    city.append('')
    PO_region.append('')
    postal_code.append('')
    country.append('')
    address1.append('')
    address2.append('')
    address3.append('')
    address4.append('')
    sa_city.append('')
    sa_region.append('')
    sa_postalcode.append('')
    sa_country.append('')
    invoice_number.append(invoice_number_input)
    ref.append('')
    invoice_date.append(invoice_date_input)
    due_date.append(invoice_date_input)
    planned_date.append('')
    tax_total.append(0)
    invoice_amount_paid.append(0)
    inv_item_code.append('')
    description.append(invoicable_desc)
    quantitiy.append(1)
    unit_amount.append(invoicable_value)
    discount.append('')
    line_amount.append(invoicable_value)
    invoice_amount_due.append(invoicable_value)
    account_code.append(get_account_no(invcoice_desc_lower))
    tax_type.append('No VAT')
    tax_amount.append(0)
    trackn1.append('')
    tracko1.append('')
    trackn2.append('')
    tracko2.append('')
    currency.append('GBP')
    type_.append('Sales Invoice')
    sent.append('')
    status.append('')

#Calculte the SUM of all the valuse on the list invoicable_value - create 2 lists total and invoice_amount_due
invoice_total_sum=0
total = []
invoice_total_sum = sum(unit_amount)
total = [invoice_total_sum] * len(currency)
print("**Successful**")


invoiceable POS - GSUMG COM £2,513.85
invoiceable POS - GSUMG COM £2,486.88
invoiceable POS - RANDR 2023-2024 ICBRates 01/05/23 to 31/03/24 M COM £3,834.83Quality Outcomes Framework 
invoiceable POS - Aspiration COM £2,217.12
invoiceable POS - QUACHG PY 2022/23 new adj value Atrial Fibrillation (AF) QOF Achvemnt A COM £11.90
invoiceable POS - MENCBG Men B - GMS NHSE £10.06
invoiceable POS - SHINGG Shingles Immunisation (GMS) NHSE £20.12
invoiceable POS - ZPERTG Pertussis (Whooping Cough) NHSE £20.12
invoiceable POS - PNEUMG Pneumoc Imm DES (GMS) NHSE £30.18
invoiceable POS - MMRVAG Measles Mumps and Rubella (MMR) Vaccinations (GMS) NHSE £40.24
invoiceable POS - CDFLUG Childhood Flu DES (GMS) NHSE £50.30
invoiceable POS - SIXVAG C&M-GMS - 6 in 1 Vaccination NHSE £50.30
invoiceable POS - INFLUG Influenza DES (GMS) NHSE £1,569.36
invoiceable POS - DESPRG GMS DES Participation in the PCN COM £592.51
invoiceable POS - ZPRCAG LBS End of Life Apr-Mar24 M COM £144.68
invoiceable POS - ZPRCAG L

# Creating **Pandas DataFrame**


In [ ]:
import pandas as pd

# dictionary of lists FUMCTION
def make_dict():

  dict = {
    'ContactName': contact_name,
    'EmailAddress': email_address,
    'POAddressLine1':PO1,
    'POAddressLine2': PO2,
    'POAddressLine3': PO3,
    'POAddressLine4': PO4,
    'POCity': city,
    'PORegion': PO_region,
    'POPostalCode': postal_code,
    'POCountry': country,
    'SAAddressLine1': address1,
    'SAAddressLine2': address2,
    'SAAddressLine3': address3,
    'SAAddressLine4': address4,
    'SACity': sa_city,
    'SARegion': sa_region,
    'SAPostalCode': sa_postalcode,
    'SACountry': sa_country,
    'InvoiceNumber': invoice_number,
    'Reference': ref,
    'InvoiceDate': invoice_date,
    'DueDate': invoice_date,
    'PlannedDate': invoice_date,
    'Total': total,
    'TaxTotal': tax_total,
    'InvoiceAmountPaid': invoice_amount_paid,
    'InvoiceAmountDue': invoice_amount_due,
    'InventoryItemCode': inv_item_code,
    'Description': description,
    'Quantity': quantitiy,
    'UnitAmount': unit_amount,
    'Discount': discount,
    'LineAmount': line_amount,
    'AccountCode': account_code,
    'TaxType': tax_type,
    'TaxAmount': tax_amount,
    'TrackingName1': trackn1,
    'TrackingOption1': tracko1,
    'TrackingName2': trackn2,
    'TrackingOption2': tracko2,
    'Currency':	currency,
    'Type':	type_,
    'Sent':	sent,
    'Status': status
  }
  return dict

df = pd.DataFrame(make_dict())
display(df.shape)

print("DataFrame **Successful**")

(29, 44)

DataFrame **Successful**


# **Mount** Google Dirve for file export.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ✋ Create a '**Xero Automation**' folder on your Google Drive.
You only need to do this once! Please note the file name is **case sensitive**.<BR>
[Open Google Drive in a new tab.](https://drive.google.com/drive/my-drive)


# Export **CSV** to Google Drive.
The file will be saved in **Xero Automation** folder on your Google Drive.

In [ ]:
file_name = "Xero Import CSV - " + str(invoice_date_input) + ".csv"
output_path = "/content/drive/MyDrive/Xero Automation/" + file_name
df.to_csv(output_path, index=False)

print("✅ File Export Successful >>>  MyGoogleDrive/Xero Automation/"  + file_name + "\n")

check_total = round(total_amount - invoice_total_sum, 2)

if check_total > 0.0:
  print(f"PDF Invoice Total = £{total_amount}")
  print(f"CSV Invoice Total = £{round(invoice_total_sum, 2)}")
  print(f"⚠️ Manually add invoice item for £{check_total} to Xero after import. Code as NHS Income Other")
else:
  print("👍 PDF Invoice Total and CSV Invoice Totals MATCHES!!")



✅ File Export Successful >>>  MyGoogleDrive/Xero Automation/Xero Import CSV - 23-Nov-2023.csv

👍 PDF Invoice Total and CSV Invoice Totals MATCHES!!
